# CIV1538: My First Logit Model
This tutorial will give you through the process of creating and estimating a discrete choice model using the Python package Biogeme. Thank you to Professor Michel Bierlaire of EPFL for making his data, code, and examples available for public use.

Your first task is to install Python on your computer for use of the Biogeme package. This can be downloaded as an executable file from the Python website for Windows and Mac machines (Linux comes preloaded with Python). You will need to install Python version 3.7+ for the easiest installation of Biogeme.

# 01logit.py Basic Biogeme Script
The first step is to import the necessary packages: Pandas, Biogeme, and headers.

In [1]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
from headers import *

The next step is to import the estimation data. Biogeme was recently updated to use the excellent package Pandas for data preparation. I import the swissmetro data as a Pandas table, then add it as a database within the internal Biogeme code structure. We can view the first few rows of the data using the command: ```database.data.head()```

In [2]:
pandas = pd.read_table("swissmetro.dat")
database = db.Database("swissmetro",pandas)
database.data.head()

GROUP  SURVEY  SP  ID  PURPOSE  FIRST  TICKET  WHO  LUGGAGE  AGE   ...    \
0      2       0   1   1        1      0       1    1        0    3   ...     
1      2       0   1   1        1      0       1    1        0    3   ...     
2      2       0   1   1        1      0       1    1        0    3   ...     
3      2       0   1   1        1      0       1    1        0    3   ...     
4      2       0   1   1        1      0       1    1        0    3   ...     

   TRAIN_TT  TRAIN_CO  TRAIN_HE  SM_TT  SM_CO  SM_HE  SM_SEATS  CAR_TT  \
0       112        48       120     63     52     20         0     117   
1       103        48        30     60     49     10         0     117   
2       130        48        60     67     58     30         0     117   
3       103        40        30     63     52     20         0      72   
4       130        36        60     63     42     20         0      90   

   CAR_CO  CHOICE  
0      65       2  
1      84       2  
2      52       2  
3      52       2  
4      84       2  

[5 rows x 28 columns]

Removing some observations can be done directly using pandas.

In [3]:
print("Size of df before removals: ", database.data.shape)
remove = (((database.data.PURPOSE != 1) & (database.data.PURPOSE != 3)) | (database.data.CHOICE == 0))
database.data.drop(database.data[remove].index,inplace=True)
print("Size of df after removals: ", database.data.shape)

Size of df before removals:  (10728, 28)
Size of df after removals:  (6768, 28)


We can then define the names and starting values for the model parameters. There are 5 parameters defined for the base Swiss Metro model. Biogeme uses a constrained optimization to reduce the search space. The Beta function is called as follows:
```python
Beta(parameter name AS str, initial value AS int, lower bound AS int, upper bound AS int, fixed AS int)
```
If there are no bounds on the parameter search space, the use may enter: ```None```. A value of ```0``` for the ```fixed``` parameter will allow Biogeme to estimate the parameter. In this case, we have only fixed the ASC for the swiss metro mode, to maintain model indentification.

In [4]:
ASC_CAR = Beta('ASC_CAR',0,None,None,0)
ASC_TRAIN = Beta('ASC_TRAIN',0,None,None,0)
ASC_SM = Beta('ASC_SM',0,None,None,1)
B_TIME = Beta('B_TIME',0,None,None,0)
B_COST = Beta('B_COST',0,None,None,0)

We can then define the corresponding variables used in the model. Biogeme has an internal set of functions that are used in the definition of variables. The user can scale a variable value or modify its value using boolean logic.

In [5]:
SM_COST =  SM_CO   * (  GA   ==  0  ) 
TRAIN_COST =  TRAIN_CO   * (  GA   ==  0  )

CAR_AV_SP =  DefineVariable('CAR_AV_SP',CAR_AV  * (  SP   !=  0  ),database)
TRAIN_AV_SP =  DefineVariable('TRAIN_AV_SP',TRAIN_AV  * (  SP   !=  0  ),database)

TRAIN_TT_SCALED = DefineVariable('TRAIN_TT_SCALED',\
                                 TRAIN_TT / 100.0,database)
TRAIN_COST_SCALED = DefineVariable('TRAIN_COST_SCALED',\
                                   TRAIN_COST / 100,database)
SM_TT_SCALED = DefineVariable('SM_TT_SCALED', SM_TT / 100.0,database)
SM_COST_SCALED = DefineVariable('SM_COST_SCALED', SM_COST / 100,database)
CAR_TT_SCALED = DefineVariable('CAR_TT_SCALED', CAR_TT / 100,database)
CAR_CO_SCALED = DefineVariable('CAR_CO_SCALED', CAR_CO / 100,database)

The next step is to define the utility function for each alternative mode.

In [6]:
V1 = ASC_TRAIN + \
     B_TIME * TRAIN_TT_SCALED + \
     B_COST * TRAIN_COST_SCALED
V2 = ASC_SM + \
     B_TIME * SM_TT_SCALED + \
     B_COST * SM_COST_SCALED
V3 = ASC_CAR + \
     B_TIME * CAR_TT_SCALED + \
     B_COST * CAR_CO_SCALED

Biogeme has a variety of built-in discrete choice models, which the user can leverage. However, it also allows the user to define their own likelihood function if it is not available in the built-in model library. In this case, we use the built-in ```bioLogLogit``` function to estimate the model.

In [7]:
# Associate utility functions with the numbering of alternatives
V = {1: V1,
     2: V2,
     3: V3}

# Associate the availability conditions with the alternatives

av = {1: TRAIN_AV_SP,
      2: SM_AV,
      3: CAR_AV_SP}

logprob = bioLogLogit(V,av,CHOICE)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "01logit"
results = biogeme.estimate()

The results of the model estimation can then be printed to the terminal.

In [8]:
# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_CAR=	-0.155
ASC_TRAIN=	-0.701
B_COST=	-1.08
B_TIME=	-1.28
              Value   Std err     t-test   p-value  Rob. Std err  Rob. t-test  \
ASC_CAR   -0.154643  0.043236  -3.576747  0.000348      0.058164    -2.658749   
ASC_TRAIN -0.701195  0.054874 -12.778246  0.000000      0.082563    -8.492878   
B_COST    -1.083788  0.051830 -20.910332  0.000000      0.068225   -15.885444   
B_TIME    -1.277878  0.056884 -22.464739  0.000000      0.104256   -12.257139   

           Rob. p-value  
ASC_CAR        0.007843  
ASC_TRAIN      0.000000  
B_COST         0.000000  
B_TIME         0.000000  


Biogeme also includes functionalities to compute elasticties for model variables. The elasticity can be calculated directly by the user, but Biogeme also includes functions for calculating derivatives for the user.

We have to redefine the model for it to run in a Jupyter notebook, so there is some repetition in the following code. Note: the parameter definitions have been slightly altered to use the calculated betas. These are stored in a dictionary and the values can be accessed through reference to the appropriate dictionary keys.

In [12]:
ASC_CAR = Beta('ASC_CAR', betas['ASC_CAR'],None,None,0)
ASC_TRAIN = Beta('ASC_TRAIN', betas['ASC_TRAIN'],None,None,0)
B_COST = Beta('B_COST', betas['B_COST'],None,None,0)
B_TIME = Beta('B_TIME', betas['B_TIME'],None,None,0)

SM_COST =  SM_CO   * (  GA   ==  0  ) 
TRAIN_COST =  TRAIN_CO   * (  GA   ==  0  )

TRAIN_TT_SCALED = TRAIN_TT  / 100.0
TRAIN_COST_SCALED =  TRAIN_COST / 100
SM_TT_SCALED = SM_TT / 100.0
SM_COST_SCALED = SM_COST / 100.0
CAR_TT_SCALED = CAR_TT / 100.0
CAR_CO_SCALED = CAR_CO / 100.0

V1 = ASC_TRAIN + \
     B_TIME * TRAIN_TT_SCALED + \
     B_COST * TRAIN_COST_SCALED
V2 = ASC_SM + \
     B_TIME * SM_TT_SCALED + \
     B_COST * SM_COST_SCALED
V3 = ASC_CAR + \
     B_TIME * CAR_TT_SCALED + \
     B_COST * CAR_CO_SCALED

# Associate utility functions with the numbering of alternatives
V = {1: V1,
     2: V2,
     3: V3}

prob1 = models.logit(V,av,1)
prob2 = models.logit(V,av,2)
prob3 = models.logit(V,av,3)

We can then calculate the set of elasticities.

Elasticities can be computed. We illustrate below two formulas. Check in the output file that they produce the same result.

First, the general definition of elasticities. This illustrates the use of the Derive expression, and can be used with any model, however complicated it is. Note the quotes in the Derive operator.

In [13]:
genelas1 = Derive(prob1,'TRAIN_TT') * TRAIN_TT / prob1
genelas2 = Derive(prob2,'SM_TT') * SM_TT / prob2
genelas3 = Derive(prob3,'CAR_TT') * CAR_TT / prob3

print(genelas1)

((Derive(exp(bioLogLogit1:((ASC_TRAIN(-0.7011954803408256) + (B_TIME(-1.2778775204302144) * (TRAIN_TT / `100.0`))) + (B_COST(-1.0837876320719746) * ((TRAIN_CO * (GA == `0`)) / `100`))),2:((ASC_SM(0) + (B_TIME(-1.2778775204302144) * (SM_TT / `100.0`))) + (B_COST(-1.0837876320719746) * ((SM_CO * (GA == `0`)) / `100.0`))),3:((ASC_CAR(-0.15464259929707586) + (B_TIME(-1.2778775204302144) * (CAR_TT / `100.0`))) + (B_COST(-1.0837876320719746) * (CAR_CO / `100.0`))))),'TRAIN_TT') * TRAIN_TT) / exp(bioLogLogit1:((ASC_TRAIN(-0.7011954803408256) + (B_TIME(-1.2778775204302144) * (TRAIN_TT / `100.0`))) + (B_COST(-1.0837876320719746) * ((TRAIN_CO * (GA == `0`)) / `100`))),2:((ASC_SM(0) + (B_TIME(-1.2778775204302144) * (SM_TT / `100.0`))) + (B_COST(-1.0837876320719746) * ((SM_CO * (GA == `0`)) / `100.0`))),3:((ASC_CAR(-0.15464259929707586) + (B_TIME(-1.2778775204302144) * (CAR_TT / `100.0`))) + (B_COST(-1.0837876320719746) * (CAR_CO / `100.0`))))))


Second, the elasticity of logit models. See Ben-Akiva and Lerman for the formula.

We can then complete the process using the .simulate() function. This will calculate the probabilities and elasticities for each observation. The Pandas .describe() function is useful to print the statistics for each of these calculated values (i.e. mean, minimum, maximum, standard deviation, percentiles).

In [14]:
logitelas1 = TRAIN_AV_SP * (1.0 - prob1) * TRAIN_TT_SCALED * B_TIME
logitelas2 = SM_AV * (1.0 - prob2) * SM_TT_SCALED * B_TIME
logitelas3 = CAR_AV_SP * (1.0 - prob3) * CAR_TT_SCALED * B_TIME

simulate = {'Prob. train': prob1,
            'Prob. Swissmetro': prob2,
            'Prob. car':prob3,
            'logit elas. 1':logitelas1,
            'generic elas. 1':genelas1,
            'logit elas. 2':logitelas2,
            'generic elas. 2':genelas2,
            'logit elas. 3':logitelas3,
            'generic elas. 3':genelas3}

biogeme  = bio.BIOGEME(database,simulate)
biogeme.modelName = "01logit_simul"
results = biogeme.simulate()
print("Results=",results.describe())

09-11-2018:16:55:57,326 WARNING  [biogeme.py:139] The chosen alternative is not available for the following observations (rownumber[choice]): 9[3]-10[3]-11[3]-12[3]-13[3]-14[3]-15[3]-16[3]-17[3]-36[3]-37[3]-38[3]-39[3]-40[3]-41[3]-42[3]-43[3]-44[3]-81[3]-82[3]-83[3]-84[3]-85[3]-86[3]-87[3]-88[3]-89[3]-90[3]-91[3]-92[3]-93[3]-94[3]-95[3]-96[3]-97[3]-98[3]-99[3]-100[3]-101[3]-102[3]-103[3]-104[3]-105[3]-106[3]-107[3]-108[3]-109[3]-110[3]-111[3]-112[3]-113[3]-114[3]-115[3]-116[3]-126[3]-127[3]-128[3]-129[3]-130[3]-131[3]-132[3]-133[3]-134[3]-180[3]-181[3]-182[3]-183[3]-184[3]-185[3]-186[3]-187[3]-188[3]-207[3]-208[3]-209[3]-210[3]-211[3]-212[3]-213[3]-214[3]-215[3]-234[3]-235[3]-236[3]-237[3]-238[3]-239[3]-240[3]-241[3]-242[3]-243[3]-244[3]-245[3]-246[3]-247[3]-248[3]-249[3]-250[3]-251[3]-279[3]-280[3]-281[3]-282[3]-283[3]-284[3]-285[3]-286[3]-287[3]-288[3]-289[3]-290[3]-291[3]-292[3]-293[3]-294[3]-295[3]-296[3]-297[3]-298[3]-299[3]-300[3]-301[3]-302[3]-303[3]-304[3]-305[3]-306[3]-307[3]-

Results=        Prob. train  Prob. Swissmetro    Prob. car  logit elas. 1  \
count  6768.000000       6768.000000  6768.000000    6768.000000   
mean      0.134159          0.604319     0.261522      -1.872641   
std       0.060524          0.182309     0.203745       0.886452   
min       0.000010          0.000281     0.000000     -13.059783   
25%       0.093416          0.487021     0.095531      -2.377340   
50%       0.128313          0.611891     0.247457      -1.769954   
75%       0.166543          0.753879     0.384109      -1.196828   
max       0.642728          0.971399     0.999710      -0.232841   

       generic elas. 1  logit elas. 2  generic elas. 2  logit elas. 3  \
count      6768.000000    6768.000000      6768.000000    6768.000000   
mean         -1.872641      -0.447852        -0.447852      -1.136721   
std           0.886452       0.498269         0.498269       1.074354   
min         -13.059783     -10.169052       -10.169052     -19.934889   
25%          

# Nested Logit Model
It is straightforward to extend the logit model to a nested specification in Biogeme. We explore a structure with existing modes (i.e. car and train) in one nest and proposed modes (i.e. swiss metro) in a separate nest. We need to define a nesting coefficient for one of the nests, with the other assumed fixed at 1.0.

In [130]:
MU = Beta('MU',2.05,1,10,0)

The utility functions are specified in the same way as for a simple multinomial logit model.

In [131]:
V1 = ASC_TRAIN + \
     B_TIME * TRAIN_TT_SCALED + \
     B_COST * TRAIN_COST_SCALED
V2 = ASC_SM + \
     B_TIME * SM_TT_SCALED + \
     B_COST * SM_COST_SCALED
V3 = ASC_CAR + \
     B_TIME * CAR_TT_SCALED + \
     B_COST * CAR_CO_SCALED

# Associate utility functions with the numbering of alternatives
V = {1: V1,
     2: V2,
     3: V3}

# Associate the availability conditions with the alternatives
CAR_AV_SP =  DefineVariable('CAR_AV_SP',CAR_AV  * (  SP   !=  0  ),database)
TRAIN_AV_SP =  DefineVariable('TRAIN_AV_SP',TRAIN_AV  * (  SP   !=  0  ),database)

av = {1: TRAIN_AV_SP,
      2: SM_AV,
      3: CAR_AV_SP}

Biogeme requires some additional details to define the nested logit models. We must define the two nests, the scale parameter associated with each nest, and the alternatives in each nest. 

In [137]:
#Definition of nests:
# 1: nests parameter
# 2: list of alternatives
existing = MU , [1,3]
future = 1.0 , [2]
nests = existing,future

              Value   Std err     t-test   p-value  Rob. Std err  Rob. t-test  \
ASC_CAR   -0.167211  0.037136  -4.502658  0.000007      0.054529    -3.066429   
ASC_TRAIN -0.511969  0.045179 -11.332138  0.000000      0.079113    -6.471350   
B_COST    -0.856634  0.046274 -18.512411  0.000000      0.060037   -14.268522   
B_TIME    -0.898627  0.056992 -15.767558  0.000000      0.107117    -8.389219   
MU         2.054209  0.117724  17.449382  0.000000      0.164242    12.507203   

           Rob. p-value  
ASC_CAR    2.166321e-03  
ASC_TRAIN  9.713119e-11  
B_COST     0.000000e+00  
B_TIME     0.000000e+00  
MU         0.000000e+00  
Results= 
Results for model [02nested]
Output file (HTML):			02nested~02.html
Nbr of parameters:		5
Sample size:			6768
Excluded data:			3960
Init log likelihood:		-6511.31
Final log likelihood:		-5236.9
Likelihood ratio test:		2548.819
Rho square:			0.196
Rho bar square:			0.195
Akaike Information Criterion:	10483.8
Bayesian Information Criterion:	10517.

The final component of the nested logit code is, again, similar to that for a multinomial logit model. The difference is that we now use the .lognested() function, which requires an additional input parameter specifying the nesting structure.

In [ ]:
# The choice model is a nested logit, with availability conditions
logprob = models.lognested(V,av,nests,CHOICE)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "02nested"
results = biogeme.estimate()
# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)
print("Results=",results)